<a href="https://colab.research.google.com/github/DavidSSalamanca/Problem-set-2/blob/main/problem_set_2_RF_y_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar XGBoost si no está instalado
!pip install xgboost


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report
# Cargar los datos de entrenamiento y prueba
train_hogares = pd.read_csv('train_hogares_final.csv')
test_hogares = pd.read_csv('test_hogares_final.csv')

# Definir las características (X) y la variable objetivo (y) del conjunto de entrenamiento
X_train = train_hogares.drop(columns=['id', 'Pobre','Dominio'])  # eliminar variables que no se van a usar
y_train = train_hogares['Pobre']  #

# Definir las características del conjunto de prueba, eliminando 'id'
X_test = test_hogares.drop(columns=['id','Dominio'])
# Identificar las columnas que están en X_train pero no en X_test
columnas_sobrantes= set(X_train.columns) - set(X_test.columns)

# Eliminar esas columnas de X_train
X_train_reducido = X_train.drop(columns=columnas_sobrantes)

# Alinear el conjunto de prueba con las columnas de X_train
X_test = X_test[X_train_reducido.columns]

# Verificar que los conjuntos tengan las mismas columnas
print(X_train_reducido.columns == X_test.columns)




[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [ ]:
seed=202013547

In [ ]:

# Definir el clasificador XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', tree_method='gpu_hist', random_state=seed, n_jobs=-1)

# Definir los hiperparámetros a probar
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.3],
    'subsample': [0.8, 1.0]
}

# Configurar GridSearchCV para encontrar los mejores parámetros maximizando F1 score
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='f1', verbose=2, n_jobs=-1)

# Entrenar el modelo con los datos de entrenamiento
grid_search.fit(X_train_reducido, y_train)

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros encontrados: ", grid_search.best_params_)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:47:26] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:47:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores parámetros encontrados:  {'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 200, 'subsample': 0.8}


In [ ]:
# Predecir en el conjunto de entrenamiento para ver el desempeño
y_train_pred = grid_search.predict(X_train_reducido)

# Evaluar el F1 score y otras métricas
print("Reporte de clasificación en el conjunto de entrenamiento:")
print(classification_report(y_train, y_train_pred))


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:48:47] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [22:48:47] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Reporte de clasificación en el conjunto de entrenamiento:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93    131936
           1       0.77      0.61      0.68     33024

    accuracy                           0.89    164960
   macro avg       0.84      0.78      0.81    164960
weighted avg       0.88      0.89      0.88    164960



In [ ]:
# Predecir sobre el conjunto de prueba con el modelo optimizado
y_test_pred = grid_search.predict(X_test)

# Crear el DataFrame para el submission en Kaggle
submission = pd.DataFrame({
    'id': test_hogares['id'],  # Mantener la columna 'id'
    'pobre': y_test_pred       # Predicciones del modelo
})

# Guardar el archivo en formato CSV para el submission en Kaggle
submission.to_csv('XGB_gamma_0,1_lr_0,1_arboles_200.csv', index=False)


In [ ]:

# Definir el clasificador Random Forest
rf = RandomForestClassifier(random_state=seed)

# Definir los hiperparámetros a probar en la búsqueda
param_grid = {
    'n_estimators': [100,200,250],
    'max_depth': [15,30, None],
    'min_samples_split': [7, 10,15],
    'min_samples_leaf': [ 7, 15,15]
}

# Configurar GridSearchCV para encontrar los mejores parámetros maximizando F1 score
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='f1', verbose=2, n_jobs=-1)

# Entrenar el modelo con los datos de entrenamiento alineados
grid_search.fit(X_train_reducido, y_train)

# Imprimir los mejores parámetros
print("Mejores parámetros encontrados: ", grid_search.best_params_)


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Mejores parámetros encontrados:  {'max_depth': None, 'min_samples_leaf': 7, 'min_samples_split': 15, 'n_estimators': 200}


In [ ]:
# Predecir en el conjunto de entrenamiento para ver el desempeño
y_train_pred = grid_search.predict(X_train_reducido)

# Evaluar el F1 score y otras métricas
print("Reporte de clasificación en el conjunto de entrenamiento:")
print(classification_report(y_train, y_train_pred))

Reporte de clasificación en el conjunto de entrenamiento:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94    131936
           1       0.86      0.61      0.71     33024

    accuracy                           0.90    164960
   macro avg       0.88      0.79      0.83    164960
weighted avg       0.90      0.90      0.90    164960



In [ ]:
# Predecir sobre el conjunto de prueba con el modelo optimizado
y_test_pred = grid_search.predict(X_test)

# Crear el DataFrame para el submission en Kaggle
submission = pd.DataFrame({
    'id': test_hogares['id'],  # Mantener la columna 'id'
    'pobre': y_test_pred       # Predicciones del modelo
})


In [ ]:
# Guardar el archivo en formato CSV para el submission en Kaggle
submission.to_csv('RF_arboles_200.csv', index=False)